In [1]:
import requests  # 서버에 http 프로토콜로 요청
from bs4 import BeautifulSoup as bs  # html 파싱
import time  # 5초간 휴식 할 때 사용
from datetime import datetime  # 오늘 연월일 추출
import pandas as pd  # 데이터 프레임 만듬
from tqdm import tqdm
import sqlite3
from sqlalchemy import create_engine
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
# import yes24_module as ym

In [2]:
# 웹 드라이버를 버전에 맞게 자동으로 다운받고 옵션을 추가해 줌
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920, 1080) # 웹 브라우저 해상도 조절
driver.get('https://www.yes24.com/Product/Category/BestSeller') # 크롤링할 웹사이트 주소
driver.find_element(By.CSS_SELECTOR, 'option[value=\'120\']').click() # 120개씩 보기

In [ ]:
year = list(range(2023, 2007, -1))
month = list(range(1, 13))
page = list(range(1, 10))
print(year)
print(month)
print(page)

In [8]:
book_list = []
for bestYear in tqdm(range(2008,2024)):
    for bestMonth in range(1,13):
        for pageNum in range(1,10):
            url = f"https://www.yes24.com/Product/Category/MonthWeekBestSeller?categoryNumber=001&pageNumber={pageNum}&pageSize=120&type=month&saleYear={bestYear}&saleMonth={bestMonth}"
            r = requests.get(url)
            soup = bs(r.text,"lxml")
            print(f"현재 {bestYear}년도 {bestMonth}월 {pageNum}페이지 크롤링중",end='\r')
            time.sleep(3)
            for book in soup.select("#yesBestList > li"):
                year = bestYear
                month = bestMonth
                book_img = book.select_one(".img_item>span>a>em>img")["data-original"] if book.select_one(".img_item>span>a>em>img") else ""
                title_h = book.select_one(".gd_res").get_text()
                title_f = book.select_one(".gd_nameF").get_text() if book.select_one(".gd_nameF") else ""
                title_m = book.select_one(".gd_name").get_text() if book.select_one(".gd_name") else "제목없음"
                title_e = book.select_one(".gd_nameE").get_text() if book.select_one(".gd_nameE") else ""
                detail_link = "https://www.yes24.com" + book.select_one(".gd_name")["href"] if book.select_one(".gd_name") else "제목없음"
                author = [i.get_text() for i in book.select(".authPub.info_auth > a")] if book.select(".authPub.info_auth > a") else []
                publisher = book.select_one(".authPub.info_pub > a").get_text() if book.select_one(".authPub.info_pub > a") else ""
                pub_date = book.select_one(".authPub.info_date").get_text() if book.select_one(".authPub.info_date") else ""
                price = book.select_one(".yes_b").get_text() if book.select_one(".yes_b") else 0
                n_reviews = book.select_one(".txC_blue").get_text() if book.select_one(".txC_blue") else 0
                review_link = book.select_one(".rating_rvCount > a")['href'] if book.select_one(".rating_rvCount > a") else ""
                rating = book.select_one("span.rating_grade > .yes_b").get_text() if book.select_one("span.rating_grade > .yes_b") else 0
                tags = [i.get_text() for i in book.select(".info_row.info_tag > .tag > a")] if book.select(".info_row.info_tag > .tag > a") else []

                book_list.append([year,month,book_img, title_h, title_f, title_m, title_e, detail_link, author, publisher, pub_date, price, n_reviews,
                     review_link, rating, tags])

  0%|                                                                                           | 0/16 [00:00<?, ?it/s]

  6%|█████                                                                           | 1/16 [11:06<2:46:32, 666.16s/it]

 12%|██████████                                                                      | 2/16 [22:11<2:35:16, 665.49s/it]

 19%|███████████████                                                                 | 3/16 [33:15<2:24:07, 665.18s/it]

 25%|████████████████████                                                            | 4/16 [44:21<2:13:04, 665.35s/it]

...

 75%|███████████████████████████████████████████████████████████▎                   | 12/16 [2:14:39<45:17, 679.44s/it]

 81%|████████████████████████████████████████████████████████████████▏              | 13/16 [2:26:16<34:14, 684.88s/it]

 88%|█████████████████████████████████████████████████████████████████████▏         | 14/16 [2:38:27<23:17, 698.94s/it]

 94%|██████████████████████████████████████████████████████████████████████████     | 15/16 [2:50:05<11:38, 698.66s/it]

100%|███████████████████████████████████████████████████████████████████████████████| 16/16 [3:02:14<00:00, 683.39s/it]


In [25]:
for i in book_list[:2]:
    print(type(i))

<class 'list'>
<class 'list'>


In [23]:
len(book_list)

188964

In [ ]:
# 혹시 몰라서 백업 ㅠ

In [26]:
import joblib

joblib.dump(book_list, 'book_list.job')

['book_list.job']

In [28]:

df = pd.DataFrame(book_list, columns=["year","month","book_img","title_h","title_f","title_m","title_e","detail_link","author","publisher","pub_date","price", "n_reviews",
                         "review_link","rating","tags"])

In [31]:
df

,year,month,book_img,title_h,title_f,title_m,title_e,detail_link,author,publisher,pub_date,price,n_reviews,review_link,rating,tags
0,2008,1,https://image.yes24.com/goods/2599166/L,[도서],,The Secret 시크릿,수 세기 동안 단 1%만이 알았던 부와 성공의 비밀,https://www.yes24.com/Product/Goods/2599166,"[론다 번, 김우열]",살림Biz,2007년 06월,"15,120",630,https://www.yes24.com/Product/Goods/2599166?Re...,7.5,"[#끌어당김, #Y2K책장소환, #크레마클럽에있어요, #탑골공원책벌레, #올해의책]"
1,2008,1,https://image.yes24.com/goods/2776013/L,[도서],,마시멜로 두번째 이야기,,https://www.yes24.com/Product/Goods/2776013,"[호아킴 데 포사다, 엘런 싱어, 공경희]",한국경제신문사(한경비피),2007년 12월,"10,800",202,https://www.yes24.com/Product/Goods/2776013?Re...,8.8,[]
2,2008,1,https://image.yes24.com/goods/2721480/L,[도서],,리버보이,,https://www.yes24.com/Product/Goods/2721480,"[팀 보울러, 정해영]",놀,2007년 10월,"8,100",234,https://www.yes24.com/Product/Goods/2721480?Re...,8.2,"[#국립어린이청소년도서관사서추천, #올해의책]"
3,2008,1,https://image.yes24.com/goods/2783642/L,[도서],,몰입 Think hard!,인생을 바꾸는 자기 혁명,https://www.yes24.com/Product/Goods/2783642,[황농문],알에이치코리아(RHK),2007년 12월,"16,200",201,https://www.yes24.com/Product/Goods/2783642?Re...,8.0,"[#올해의책, #생활습관]"
4,2008,1,https://image.yes24.com/goods/2752350/L,[도서],,즐거운 나의 집,,https://www.yes24.com/Product/Goods/2752350,[공지영],푸른숲,2007년 11월,"8,820",328,https://www.yes24.com/Product/Goods/2752350?Re...,8.9,[#올해의책]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188959,2023,12,https://image.yes24.com/goods/117650984/L,[도서],,초수익 성장주 투자,세계에서 가장 비밀스러운 주식 투자 시스템,https://www.yes24.com/Product/Goods/117650984,"[마크 미너비니, 김태훈, 김대현]",이레미디어,2023년 03월,"22,500",49,https://www.yes24.com/Product/Goods/117650984?...,9.6,[#크레마클럽에있어요]
188960,2023,12,https://image.yes24.com/goods/117585533/L,[도서],,상처 주는 말 하는 친구에게 똑똑하게 말하는 법,"화내지 않고, 참지 않고, 울지 않는 마법의 표현 59",https://www.yes24.com/Product/Goods/117585533,"[김윤나, 유영근]",북라이프,2023년 02월,"11,700",52,https://www.yes24.com/Product/Goods/117585533?...,9.9,"[#학교생활이야기, #슬기로운언어생활, #친구관계, #인간관계에대하여]"
188961,2023,12,https://image.yes24.com/goods/115033554/L,[도서],,사람을 얻는 지혜,국내 최초 스페인어 완역본,https://www.yes24.com/Product/Goods/115033554,"[그라시안 이 모랄레스 발타사르, 김유경]",현대지성,2022년 10월,"10,800",103,https://www.yes24.com/Product/Goods/115033554?...,9.6,[#크레마클럽에있어요]
188962,2023,12,https://image.yes24.com/goods/114848694/L,[도서],,인생의 역사,‘공무도하가’에서 ‘사랑의 발명’까지,https://www.yes24.com/Product/Goods/114848694,[신형철],난다,2022년 10월,"16,200",111,https://www.yes24.com/Product/Goods/114848694?...,9.6,"[#2023서울국제도서전, #YES24XEBS_대국민독서챌린지, #2022내맘대로올..."


In [12]:
pwd

'c:\\bigdata\\self_practice'

request로 베스트셀러 페이지 수집완료!

In [30]:
df.to_csv("../data/yes24_best2008.01_2023.11.csv", index=False)

셀레니움으로 세부데이터 수집하기 (리뷰포함)

In [32]:
pd.read_csv("../data/yes24_best2008.01_2023.11.csv")

C:\Users\hi\AppData\Local\Temp\ipykernel_9944\2883447807.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("../data/yes24_best2008.01_2023.11.csv")


,year,month,book_img,title_h,title_f,title_m,title_e,detail_link,author,publisher,pub_date,price,n_reviews,review_link,rating,tags
0,2008,1,https://image.yes24.com/goods/2599166/L,[도서],NaN,The Secret 시크릿,수 세기 동안 단 1%만이 알았던 부와 성공의 비밀,https://www.yes24.com/Product/Goods/2599166,"['론다 번', '김우열']",살림Biz,2007년 06월,"15,120",630,https://www.yes24.com/Product/Goods/2599166?Re...,7.5,"['#끌어당김', '#Y2K책장소환', '#크레마클럽에있어요', '#탑골공원책벌레'..."
1,2008,1,https://image.yes24.com/goods/2776013/L,[도서],NaN,마시멜로 두번째 이야기,NaN,https://www.yes24.com/Product/Goods/2776013,"['호아킴 데 포사다', '엘런 싱어', '공경희']",한국경제신문사(한경비피),2007년 12월,"10,800",202,https://www.yes24.com/Product/Goods/2776013?Re...,8.8,[]
2,2008,1,https://image.yes24.com/goods/2721480/L,[도서],NaN,리버보이,NaN,https://www.yes24.com/Product/Goods/2721480,"['팀 보울러', '정해영']",놀,2007년 10월,"8,100",234,https://www.yes24.com/Product/Goods/2721480?Re...,8.2,"['#국립어린이청소년도서관사서추천', '#올해의책']"
3,2008,1,https://image.yes24.com/goods/2783642/L,[도서],NaN,몰입 Think hard!,인생을 바꾸는 자기 혁명,https://www.yes24.com/Product/Goods/2783642,['황농문'],알에이치코리아(RHK),2007년 12월,"16,200",201,https://www.yes24.com/Product/Goods/2783642?Re...,8.0,"['#올해의책', '#생활습관']"
4,2008,1,https://image.yes24.com/goods/2752350/L,[도서],NaN,즐거운 나의 집,NaN,https://www.yes24.com/Product/Goods/2752350,['공지영'],푸른숲,2007년 11월,"8,820",328,https://www.yes24.com/Product/Goods/2752350?Re...,8.9,['#올해의책']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188959,2023,12,https://image.yes24.com/goods/117650984/L,[도서],NaN,초수익 성장주 투자,세계에서 가장 비밀스러운 주식 투자 시스템,https://www.yes24.com/Product/Goods/117650984,"['마크 미너비니', '김태훈', '김대현']",이레미디어,2023년 03월,"22,500",49,https://www.yes24.com/Product/Goods/117650984?...,9.6,['#크레마클럽에있어요']
188960,2023,12,https://image.yes24.com/goods/117585533/L,[도서],NaN,상처 주는 말 하는 친구에게 똑똑하게 말하는 법,"화내지 않고, 참지 않고, 울지 않는 마법의 표현 59",https://www.yes24.com/Product/Goods/117585533,"['김윤나', '유영근']",북라이프,2023년 02월,"11,700",52,https://www.yes24.com/Product/Goods/117585533?...,9.9,"['#학교생활이야기', '#슬기로운언어생활', '#친구관계', '#인간관계에대하여']"
188961,2023,12,https://image.yes24.com/goods/115033554/L,[도서],NaN,사람을 얻는 지혜,국내 최초 스페인어 완역본,https://www.yes24.com/Product/Goods/115033554,"['그라시안 이 모랄레스 발타사르', '김유경']",현대지성,2022년 10월,"10,800",103,https://www.yes24.com/Product/Goods/115033554?...,9.6,['#크레마클럽에있어요']
188962,2023,12,https://image.yes24.com/goods/114848694/L,[도서],NaN,인생의 역사,‘공무도하가’에서 ‘사랑의 발명’까지,https://www.yes24.com/Product/Goods/114848694,['신형철'],난다,2022년 10월,"16,200",111,https://www.yes24.com/Product/Goods/114848694?...,9.6,"['#2023서울국제도서전', '#YES24XEBS_대국민독서챌린지', '#2022..."


In [33]:
df['detail_link']

0           https://www.yes24.com/Product/Goods/2599166
1           https://www.yes24.com/Product/Goods/2776013
2           https://www.yes24.com/Product/Goods/2721480
3           https://www.yes24.com/Product/Goods/2783642
4           https://www.yes24.com/Product/Goods/2752350
                              ...                      
188959    https://www.yes24.com/Product/Goods/117650984
188960    https://www.yes24.com/Product/Goods/117585533
188961    https://www.yes24.com/Product/Goods/115033554
188962    https://www.yes24.com/Product/Goods/114848694
188963    https://www.yes24.com/Product/Goods/113416787
Name: detail_link, Length: 188964, dtype: object

In [ ]:
detail_pages = [] 
for links in data["detail_link"][:5]:
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.set_window_size(1920, 1080) # 웹 브라우저 해상도 조절
    driver.get(links)
    time.sleep(3)
    
    try:
        while True:
            more_box2 = driver.find_element(By.CSS_SELECTOR, "#storeView > div.container > div.store_review > a")

            more_box2.click()
            time.sleep(2)  # 클릭 후 로딩을 위한 대기 시간 (필요에 따라 조절)

    except: 
        page_source2 = driver.page_source
        detail_pages.append(page_source2)
        driver.quit()

In [36]:
driver.get("https://www.yes24.com/Product/Goods/113416787")

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.200)
Stacktrace:
	GetHandleVerifier [0x003C72A3+45731]
	(No symbol) [0x00352D51]
	(No symbol) [0x0024880D]
	(No symbol) [0x0022F75E]
	(No symbol) [0x0029C11B]
	(No symbol) [0x002AB2D3]
	(No symbol) [0x00297DD6]
	(No symbol) [0x002731F6]
	(No symbol) [0x0027439D]
	GetHandleVerifier [0x006D0716+3229462]
	GetHandleVerifier [0x007184C8+3523784]
	GetHandleVerifier [0x0071214C+3498316]
	GetHandleVerifier [0x00451680+611968]
	(No symbol) [0x0035CCCC]
	(No symbol) [0x00358DF8]
	(No symbol) [0x00358F1D]
	(No symbol) [0x0034B2C7]
	BaseThreadInitThunk [0x777BFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77CF7C6E+286]
	RtlGetAppContainerNamedObjectPath [0x77CF7C3E+238]
